In [1]:
#Bibliotecas 
import pandas as pd
import openpyxl
import os
from datetime import datetime
import numpy as np
import psycopg2 as pg
import configparser
import pygwalker

In [2]:
#Hora de início
today = pd.Timestamp(datetime.today())
today

Timestamp('2024-10-07 13:25:03.637055')

In [3]:
#Path credenciais
crendenciais_path = r'\\glwfs02.lwart.net\LWC-FLORESTAL\Qualidade_Florestal\03- ADMINISTRATIVO\2023\06- COLABORADORES\Gabriel\18 - SQL\sets.conf'

In [4]:
#Path premissas
path_premissas = r'\\GLWFS02.lwart.net\LWC-FLORESTAL\Qualidade_Florestal\03- ADMINISTRATIVO\2023\06- COLABORADORES\Gabriel\19 - Projeto SOP\premissas.xlsx'

In [5]:
#Path justificativas
path_justificativas = r'\\glwfs02.lwart.net\LWC-FLORESTAL\Qualidade_Florestal\03- ADMINISTRATIVO\2023\06- COLABORADORES\Gabriel\19 - Projeto SOP\justificativas.xlsx'

In [6]:
#Path Weed

path_weed = r'F:\Planejamento_e_Controle\Geoprocessamento\02_SETOR\06_SENSORIAMENTO REMOTO\PROJETOS\03_MATO-COMPETICAO\13_PLANILHA_BI\Planilha_WEED_COMPETITION_BI.xlsx'

In [7]:
#Path boletim
df_boletim1 = r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2020_Boletim_da_Silvicultura.parquet'
df_boletim2 = r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2021_Boletim_da_Silvicultura.parquet'
df_boletim3 = r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2022_Boletim_da_Silvicultura.parquet'
df_boletim4 = r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2023_Boletim_da_Silvicultura.parquet'
df_boletim5 =r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2024_Boletim_da_Silvicultura.parquet'

In [8]:
#Função máscara

def mascara(df, coluna,n_digitos):
    df[coluna] = df[coluna].astype(str)
    df[coluna] = str("000") + df[coluna]
    df[coluna] = df[coluna].astype(str).str[-n_digitos:] 
    return df

In [9]:
#Credenciais

credenciais = configparser.ConfigParser()

credenciais.read(crendenciais_path)

logins = credenciais['informacoes']



conn_dwh = pg.connect(
    user = logins['login'],
    password = logins['senha'],
    host = logins['host'],
    port = logins['port'],
    database = logins['data_base']
    )

In [10]:
#Cadastro


cursor = conn_dwh.cursor()



cursor.execute("""
               
SELECT 
    f.data_plantio,
    f.cd_regiao,
    f.id_projeto,
    f.cd_projeto,
    f.cd_talhao,
    f.num_ciclo,
    f.cd_material_genetico,
    f.cd_regime,
    f.num_rotacao,
    f.vlr_area,
    f.source,
    f.data_reg,
    f.est_reg,
    f.cd_uso_solo,
    f.cd_caracteristica_uso_solo,
    f.cd_espacamento
FROM 
    forestry.f_cfg_uso_solo f


WHERE 
    f.source = 'BSP' 
    AND f.tip_reg = 'A' 
    AND f.est_reg = 'A'           
    AND f.cd_regiao IN ('1','2')

               """)


dados_consulta = cursor.fetchall()

cursor.close()
conn_dwh.close()


colunas_cad = [desc[0] for desc in cursor.description]


cadastro = pd.DataFrame(dados_consulta,columns=colunas_cad)
cadastro = mascara(cadastro,'id_projeto', 4)
cadastro = mascara(cadastro,'cd_talhao', 3)
cadastro['objeto de locação'] = cadastro['id_projeto'] + cadastro['cd_talhao']
cadastro = cadastro.drop_duplicates(subset='objeto de locação', keep='first')

cadastro_type = {'data_plantio': 'datetime64[ns]',
                 'vlr_area' : 'float'}

cadastro = cadastro.astype(cadastro_type)

In [11]:
#De para Operação

operacao = pd.read_excel(path_premissas, sheet_name='operacao',usecols=['Operações', 'Grupo de Operação','Weed'])

operacao = operacao[operacao['Weed'] == 'Sim']


In [12]:
#De para status

status = pd.read_excel(path_premissas,sheet_name='status')


In [13]:
#De para classe
classe_intensidade = pd.read_excel(path_premissas, sheet_name='Classe weed')


In [14]:
#Boletim

boletim1 = pd.read_parquet(df_boletim1, engine='fastparquet')
boletim2 = pd.read_parquet(df_boletim2, engine='fastparquet')
boletim3 = pd.read_parquet(df_boletim3, engine='fastparquet')
boletim4 = pd.read_parquet(df_boletim4, engine='fastparquet')
boletim5 = pd.read_parquet(df_boletim5, engine='fastparquet')

boletim = pd.concat([boletim1, boletim2, boletim3, boletim4, boletim5])
boletim = boletim[['ID_REGIAO','CD_BOLETIM_SILVICULTURA','DATA_OPERACAO','VLR_PRODUCAO','DCR_OPERACAO','CD_TALHAO','ID_PROJETO','CD_PROJETO','NOME_RECURSO','QUANTIDADE','UNIDADE_MED','DCR_FASE_PRESCRICAO','FLAG_OCORRENCIA','NOM_PROJETO','TIP_APROVACAO','CD_MATERIAL_GENETICO']]


In [15]:
#Boletim; Operação; Status 

#Operações cadastradas
boletim = boletim.merge(operacao, left_on='DCR_OPERACAO', right_on='Operações', how='left')

boletim = boletim[boletim['Weed'] == 'Sim']


#Status
boletim = boletim.merge(status, left_on='TIP_APROVACAO', right_on='sigla', how='left')

boletim = boletim[boletim['Status Vigente'] == 'Sim']





In [16]:
# % de Área realizada 

area_realizada = 0

In [17]:
#Tratando o Boletim

boletim = mascara(boletim,'ID_PROJETO',4)

boletim = mascara(boletim,'CD_TALHAO',3)

boletim['objeto de locação'] = boletim['ID_PROJETO'] + boletim['CD_TALHAO']

boletim['Dose/ha'] = round((boletim['QUANTIDADE'] / boletim['VLR_PRODUCAO']),3)

boletim = boletim.sort_values(['DATA_OPERACAO','CD_BOLETIM_SILVICULTURA','QUANTIDADE'], ascending=[False, True, True])

boletim['insumos apontados'] = boletim.groupby('CD_BOLETIM_SILVICULTURA')['NOME_RECURSO'].transform(lambda x: '; '.join(x.astype(str)))

boletim['doses apontadas'] = boletim.groupby('CD_BOLETIM_SILVICULTURA')['Dose/ha'].transform(lambda x: '; '.join(x.astype(str)))


boletim = boletim.drop_duplicates(subset=['CD_BOLETIM_SILVICULTURA'], keep='first')

#Cadastro

boletim = boletim.merge(cadastro[['objeto de locação','vlr_area']], left_on='objeto de locação', right_on='objeto de locação', how='left')

boletim['% area realizada'] = round((boletim['vlr_area'] / boletim['VLR_PRODUCAO']),3)


In [18]:
#Filtro % de área apontada

percent_area = 0

boletim = boletim[boletim['% area realizada'] >= percent_area]



In [19]:
#Weed

weed = pd.read_excel(path_weed, sheet_name='Planilha1')

weed = weed[['ID_PROJETO', 'PROJETO', 'ID_TALHAO', 'AREA_HA', 'IDADE', 'DATA_PLANT', 'ESTADO', 'DATA_IMG','DATA_IMG_NOVA','Matocompetição', 'Plantio Limpo',
             'Nuvem', 'Classe Idade', 'Alerta', '% DA ÁREA TOTAL','Classe_Area', 'Regional Silvicultura', 'Período', 'Data de entrega']]

weed = weed[weed['ESTADO'] != 'MS']
weed = mascara(weed,'ID_PROJETO',4)
weed = mascara(weed,'ID_TALHAO',3)

weed['objeto de locação'] = weed['ID_PROJETO'] + weed['ID_TALHAO']

weed = weed.sort_values(['objeto de locação', 'DATA_IMG'], ascending=[True, False])


weed = weed.sort_values(['objeto de locação', 'DATA_IMG'], ascending=[True, False])

weed['DATA_IMG_NOVA'] = weed.apply(lambda x: x['DATA_IMG'] if pd.isna(x['DATA_IMG_NOVA']) else x['DATA_IMG_NOVA'],axis=1)

weed['quinzena'] = np.where(weed['DATA_IMG_NOVA'].dt.day < 15,'1ª',
                                 np.where(weed['DATA_IMG_NOVA'].dt.day > 15, '2ª', ''))


weed = weed.merge(classe_intensidade, left_on=['Classe_Area'], right_on=['Classe Weed'], how='left').drop('Classe Weed',axis =1)


weed['ordem imagem'] = weed.groupby('objeto de locação')['objeto de locação'].transform('size') - weed.groupby('objeto de locação').cumcount()


weed['intensidade'] = np.where(
weed['ordem imagem'] == 1 , '1º Monitoramento',
np.where(
(weed['objeto de locação'] == weed['objeto de locação'].shift(-1)) & (weed['Intensidade'] > weed['Intensidade'].shift(-1)) , 'Aumentou a intensidade', 
np.where(
(weed['objeto de locação'] == weed['objeto de locação'].shift(-1)) & (weed['Intensidade'] < weed['Intensidade'].shift(-1)) , 'Reduziu a intensidade', 
'Sem alterações de intensidade' )))


In [20]:
#Weed e Boletim

weed = weed.merge(boletim[['CD_BOLETIM_SILVICULTURA','DATA_OPERACAO','insumos apontados', 'doses apontadas','% area realizada','objeto de locação', 'DCR_OPERACAO','Grupo de Operação']], left_on='objeto de locação', right_on='objeto de locação', how='left')


In [21]:
#Calculos do weed
intervalo_min = -20

weed['intervalo imagem operação'] = (weed['DATA_OPERACAO'] - weed['DATA_IMG_NOVA']).dt.days

weed['intervalo imagem operação abs'] = np.abs(weed['intervalo imagem operação'])

weed['Alerta'] = weed['Alerta'].str.upper()


weed['status'] = np.where( (weed['Alerta'] == 'ALERTA') & (pd.isna(weed['DATA_OPERACAO'])) ,
                          'Não realizado', 
                          np.where((weed['Alerta'] == 'ALERTA') & (pd.notna(weed['DATA_OPERACAO'])),
                                   'Realizado',
                                np.where((weed['Alerta'] == 'SEM ALERTA'), 'Sem necessidade',
                                    ''
                                    )))


weed['intervalo'] = (weed['intervalo imagem operação'] >= intervalo_min) | (weed['status'] != 'Não realizado')

weed = weed[weed['intervalo'] == True]

weed['chave_ordem'] = weed['objeto de locação'] + " - " + weed['ordem imagem'].astype(str)

weed = weed.sort_values(['objeto de locação','ordem imagem', 'intervalo imagem operação abs'], ascending=[True,False,True])


delta_priorizacao = 10

weed['priorização'] = np.where((
    (weed['chave_ordem'] == weed['chave_ordem'].shift(1)) &\
    ((weed['Grupo de Operação'] == 'Herbicida') & (weed['Grupo de Operação'].shift(1) == 'Roçada')) &\
    ((weed['intervalo imagem operação abs'] - weed['intervalo imagem operação abs'].shift(1)) <= delta_priorizacao) ) |\
        ((weed['chave_ordem'] == weed['chave_ordem'].shift(1)) &\
        ((weed['Grupo de Operação'] == 'Roçada') & (weed['Grupo de Operação'].shift(1) == 'Roçada')) &\
        ((weed['intervalo imagem operação abs'] - weed['intervalo imagem operação abs'].shift(1)) <= delta_priorizacao)) |\
            ((weed['chave_ordem'] == weed['chave_ordem'].shift(1)) &\
            ((weed['Grupo de Operação'] == 'Herbicida') & (weed['Grupo de Operação'].shift(1) == 'Herbicida')) &\
            ((weed['intervalo imagem operação abs'] - weed['intervalo imagem operação abs'].shift(1)) <= delta_priorizacao)) |\
(
    (weed['chave_ordem'] == weed['chave_ordem'].shift(-1)) &\
    ((weed['Grupo de Operação'] == 'Herbicida') & (weed['Grupo de Operação'].shift(-1) == 'Roçada')) &\
    ((weed['intervalo imagem operação abs'] - weed['intervalo imagem operação abs'].shift(-1)) <= delta_priorizacao) ) |\
        ((weed['chave_ordem'] == weed['chave_ordem'].shift(-1)) &\
        ((weed['Grupo de Operação'] == 'Roçada') & (weed['Grupo de Operação'].shift(-1) == 'Roçada')) &\
        ((weed['intervalo imagem operação abs'] - weed['intervalo imagem operação abs'].shift(-1)) <= delta_priorizacao)) |\
            ((weed['chave_ordem'] == weed['chave_ordem'].shift(-1)) &\
            ((weed['Grupo de Operação'] == 'Herbicida') & (weed['Grupo de Operação'].shift(-1) == 'Herbicida')) &\
            ((weed['intervalo imagem operação abs'] - weed['intervalo imagem operação abs'].shift(-1)) <= delta_priorizacao)) 
            ,'Considerar operação', 
        
                    np.where(pd.isna(weed['DATA_OPERACAO']), 'Sem operação', 'Não considerar operação'
                                ))

weed = weed[weed['priorização'] != 'Não considerar operação']

weed = weed.drop_duplicates(subset='chave_ordem', keep='first')

weed['soma alertas'] = weed.groupby('objeto de locação')['Alerta'].transform(lambda x: (x == 'ALERTA').sum())

weed['numero de imagens'] = weed.groupby('objeto de locação')['objeto de locação'].transform('count')



In [22]:
#Soma da quantidade de dias no sujo

weed['Tempo no sujo'] = np.where( (weed['objeto de locação'] == weed['objeto de locação'].shift(-1)) & (weed['Alerta'] == 'ALERTA'),
                                  (weed['DATA_IMG_NOVA'] - weed['DATA_IMG_NOVA'].shift(-1)) .dt.days,
                                np.where( (weed['objeto de locação'] == weed['objeto de locação'].shift(1)) &\
                                        ((weed['Alerta'] == 'ALERTA') & (weed['Alerta'] == 'SEM ALERTA').shift(1)) & (weed['ordem imagem'] == 1) ,15 , np.nan ))


In [23]:
#Assertividade

#Só pode ser considerada para os talhões que tem alerta

assertividade = 35


weed['assertividade'] = np.where( (weed['Alerta'] == 'SEM ALERTA') | (weed['status'] == 'Sem necessidade'), "", 
                            np.where( (weed['Alerta'] == 'ALERTA') & (weed['status'] =='Realizado') & (weed['intervalo imagem operação abs'] <= assertividade),
                                    "No prazo", "Fora do prazo")
)


In [24]:
#Boletim efetivo

#resposta para a pergunta qual boletim resolveu o alerta?

weed['boletim efetivo'] = np.where( weed['numero de imagens'] == 1, 
                                    weed['CD_BOLETIM_SILVICULTURA'],
                                np.where(
                                            ((weed['objeto de locação'] == weed['objeto de locação'].shift(-1)) | weed['ordem imagem'] == 1) &\
                                            (pd.notna(weed['DATA_OPERACAO'])) &\
                                            (weed['Alerta'] == 'ALERTA') &\
                                            ((weed['DATA_OPERACAO'] >= weed['DATA_OPERACAO'].shift(-1)) | weed['ordem imagem'] == 1) &\
                                            ((weed['Alerta'] == 'ALERTA') & (weed['Alerta'].shift(1) == 'SEM ALERTA')), 
                                            weed['CD_BOLETIM_SILVICULTURA'], np.nan ))

In [25]:
weed.columns

Index(['ID_PROJETO', 'PROJETO', 'ID_TALHAO', 'AREA_HA', 'IDADE', 'DATA_PLANT',
       'ESTADO', 'DATA_IMG', 'DATA_IMG_NOVA', 'Matocompetição',
       'Plantio Limpo', 'Nuvem', 'Classe Idade', 'Alerta', '% DA ÁREA TOTAL',
       'Classe_Area', 'Regional Silvicultura', 'Período', 'Data de entrega',
       'objeto de locação', 'quinzena', 'Intensidade', 'ordem imagem',
       'intensidade', 'CD_BOLETIM_SILVICULTURA', 'DATA_OPERACAO',
       'insumos apontados', 'doses apontadas', '% area realizada',
       'DCR_OPERACAO', 'Grupo de Operação', 'intervalo imagem operação',
       'intervalo imagem operação abs', 'status', 'intervalo', 'chave_ordem',
       'priorização', 'soma alertas', 'numero de imagens', 'Tempo no sujo',
       'assertividade', 'boletim efetivo'],
      dtype='object')

In [26]:
#Percentual de efetividade

# weed['eficiencia'] = np.where((weed['Alerta'] == 'ALERTA') &\
#                               (weed['ordem imagem'] != weed['numero de imagens']) &\
#                               (weed['ordem imagem'] != 1 ) &\
#                               (weed['objeto de locação'] == weed['objeto de locação'].shift(-1))&\
#                               (weed['CD_BOLETIM_SILVICULTURA'] != weed['CD_BOLETIM_SILVICULTURA'].shift(-1))&\
#                               (weed['Alerta'].shift(1) == 'SEM ALERTA')

#                             ,'considerado', 'nao considerado'
#                               )

In [27]:
#Criando base auxiliar para encontrar informações da eficiencia da operação

weed_eo = weed[['objeto de locação','chave_ordem','CD_BOLETIM_SILVICULTURA','Alerta','DATA_IMG_NOVA','DATA_OPERACAO']]

#Filtrar o quem data de operação vazio

weed_eo = weed_eo[pd.notna(weed_eo['DATA_OPERACAO'])]

weed_eo['DATA_OPERACAO'] = weed_eo['DATA_OPERACAO'].astype('datetime64[ns]')
weed_eo['DATA_IMG_NOVA'] = weed_eo['DATA_IMG_NOVA'].astype('datetime64[ns]')

In [28]:
weed.shape[0]

38994

In [29]:
#Base auxiliar para o weed

#Esse merge irá duplicar a base, dessa forma é necessário uma regra para filtrar somente o que deve ser considerado
weed = weed.merge(weed_eo, left_on='objeto de locação', right_on='objeto de locação', how='left', suffixes=("","_EO"))


In [30]:
weed['dist img_op'] = (weed['DATA_OPERACAO_EO'] - weed['DATA_IMG_NOVA']).dt.days

In [31]:
weed.shape[0]

402460

In [33]:
#weed[weed['objeto de locação'] == '0011018'].to_excel('weed.xlsx')